## Building A Chatbot
In this video We'll go over an example of how to design and implement an LLM-powered chatbot. This chatbot will be able to have a conversation and remember previous interactions.

Note that this chatbot that we build will only use the language model to have a conversation. There are several other related concepts that you may be looking for:

- Conversational RAG: Enable a chatbot experience over an external source of data
- Agents: Build a chatbot that can take actions

This video tutorial will cover the basics which will be helpful for those two more advanced topics.

In [32]:
import os
from dotenv import load_dotenv
load_dotenv() ## aloading all the environment variable

groq_api_key=os.getenv("GROQ_API_KEY")
groq_api_key



'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'

In [ ]:
from langchain_groq import ChatGroq
model=ChatGroq(model="llama-3.1-8b-instant",groq_api_key=groq_api_key)
model

/Users/sauravyadav/Downloads/GenerativeAI/1-Basics+Of+Langchain/.conda/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x1511bcf50>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x151557560>, model_name='llama-3.1-8b-instant', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [3]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="Hi , My name is Krish and I am a Chief AI Engineer")])

AIMessage(content="Nice to meet you, Krish. As a Chief AI Engineer, I'm sure you have a very interesting and challenging role. What specific areas of AI are you most interested in or working on currently? Are you into development, deployment, or perhaps research and strategy?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 54, 'prompt_tokens': 48, 'total_tokens': 102, 'completion_time': 0.075053604, 'prompt_time': 0.002499851, 'queue_time': 0.055204509, 'total_time': 0.077553455}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_4387d3edbb', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--fd02d1ae-eace-4f09-a2a5-f893f268df05-0', usage_metadata={'input_tokens': 48, 'output_tokens': 54, 'total_tokens': 102})

In [4]:
from langchain_core.messages import AIMessage
model.invoke(
    [
        HumanMessage(content="Hi , My name is Krish and I am a Chief AI Engineer"),
        AIMessage(content="Hello Krish! It's nice to meet you. \n\nAs a Chief AI Engineer, what kind of projects are you working on these days? \n\nI'm always eager to learn more about the exciting work being done in the field of AI.\n"),
        HumanMessage(content="Hey What's my name and what do I do?")
    ]
)

AIMessage(content="Your name is Krish, and you're a Chief AI Engineer.", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 117, 'total_tokens': 131, 'completion_time': 0.015982717, 'prompt_time': 0.007786582, 'queue_time': 0.054912808, 'total_time': 0.023769299}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_f757f4b0bf', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--1f02145b-828f-4ecd-8c5b-1a7e9a5f88d9-0', usage_metadata={'input_tokens': 117, 'output_tokens': 14, 'total_tokens': 131})

### Message History
We can use a Message History class to wrap our model and make it stateful. This will keep track of inputs and outputs of the model, and store them in some datastore. Future interactions will then load those messages and pass them into the chain as part of the input. Let's see how to use this!

In [5]:
!pip install langchain_community

In [6]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store={}
# BaseChatMessageHistory is the return type. Abstract base class for storing chat message history
def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        # ChatMessageHistory: Stores messages in memory (Python list). Implements BaseChatMessageHistory.
        store[session_id]=ChatMessageHistory()
    return store[session_id]
# Wraps a model/chain and injects session-based memory using a history provider.
with_message_history=RunnableWithMessageHistory(model,get_session_history)

In [7]:
# config for session id.
config={"configurable":{"session_id":"chat1"}}

In [8]:
response=with_message_history.invoke(
    [HumanMessage(content="Hi , My name is Krish and I am a Chief AI Engineer")],
    config=config
)

In [9]:
response.content

"Nice to meet you, Krish! As a Chief AI Engineer, that's a fascinating role. What specific areas of AI are you currently working on, and what kind of projects are you leading or contributing to?"

In [10]:
with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

AIMessage(content="Your name is Krish. You're a Chief AI Engineer.", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 105, 'total_tokens': 118, 'completion_time': 0.023065317, 'prompt_time': 0.005757117, 'queue_time': 0.055240823, 'total_time': 0.028822434}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_4387d3edbb', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--759c5c39-6af3-4c5c-9b60-1f0d01579b1c-0', usage_metadata={'input_tokens': 105, 'output_tokens': 13, 'total_tokens': 118})

In [11]:
## change the config-->session id
config1={"configurable":{"session_id":"chat2"}}
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

"I don't have any information about your name. I'm a large language model, I don't have the ability to retain personal information or recall previous conversations. Each time you interact with me, it's a new conversation. If you'd like to share your name with me, I'd be happy to chat with you!"

In [12]:
response=with_message_history.invoke(
    [HumanMessage(content="Hey My name is John")],
    config=config1
)
response.content

"Hi John. It's nice to meet you. Is there something I can help you with or would you like to chat?"

In [13]:
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

'Your name is John.'

### Prompt templates
Prompt Templates help to turn raw user information into a format that the LLM can work with. In this case, the raw user input is just a message, which we are passing to the LLM. Let's now make that a bit more complicated. First, let's add in a system message with some custom instructions (but still taking messages as input). Next, we'll add in more input besides just the messages.

In [14]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
prompt=ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful assistant.Amnswer all the question to the nest of your ability"),
        # Dynamically includes past messages.
        # When you invoke the chain later, you must pass a variable called messages.
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain=prompt|model

In [15]:
# All the messages will included in MessagesPlaceholder.
chain.invoke({"messages":[HumanMessage(content="Hi My name is Krish")]})

AIMessage(content='Nice to meet you, Krish. Is there something I can help you with or would you like to chat?', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 57, 'total_tokens': 80, 'completion_time': 0.026063988, 'prompt_time': 0.002762396, 'queue_time': 0.050372774, 'total_time': 0.028826384}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_4387d3edbb', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--137d9193-e2bf-4f2e-86d3-5e5520fc2e45-0', usage_metadata={'input_tokens': 57, 'output_tokens': 23, 'total_tokens': 80})

In [16]:
with_message_history=RunnableWithMessageHistory(chain,get_session_history)

In [17]:
config = {"configurable": {"session_id": "chat3"}}
response=with_message_history.invoke(
    [HumanMessage(content="Hi My name is Krish")],
    config=config
)

response

AIMessage(content="Nice to meet you, Krish. I'm here to help and assist you with any questions or topics you'd like to discuss. What's on your mind today? How are you doing?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 39, 'prompt_tokens': 57, 'total_tokens': 96, 'completion_time': 0.060630019, 'prompt_time': 0.00298345, 'queue_time': 0.05058415, 'total_time': 0.063613469}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_ff2b098aaf', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--29ca3c46-83a4-4d44-be8f-9070d47a1b66-0', usage_metadata={'input_tokens': 57, 'output_tokens': 39, 'total_tokens': 96})

In [18]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

'Your name is Krish.'

In [19]:
## Add more complexity

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [20]:
response=chain.invoke({"messages":[HumanMessage(content="Hi My name is Krish")],"language":"Hindi"})
response.content

'नमस्ते कृष, मैं आपकी सहायता करने के लिए यहाँ हूँ। क्या मैं आपके किसी विशेष सवाल का जवाब दे सकता हूँ या क्या आपको कोई जानकारी चाहिए?'

Let's now wrap this more complicated chain in a Message History class. This time, because there are multiple keys in the input, we need to specify the correct key to use to save the chat history.

In [21]:
with_message_history=RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)

In [22]:
config = {"configurable": {"session_id": "chat4"}}
repsonse=with_message_history.invoke(
    {'messages': [HumanMessage(content="Hi,I am Krish")],"language":"Hindi"},
    config=config
)
repsonse.content

'नमस्ते कृष, मैं आपकी मदद करने के लिए तैयार हूँ। क्या आपका कोई सवाल है या क्या मुझे आपकी किसी चीज़ में सहायता करनी होगी?'

In [23]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="whats my name?")], "language": "Hindi"},
    config=config,
)

In [24]:
response.content

'आपका नाम कृष है।'

### Managing the Conversation History
One important concept to understand when building chatbots is how to manage conversation history. If left unmanaged, the list of messages will grow unbounded and potentially overflow the context window of the LLM. Therefore, it is important to add a step that limits the size of the messages you are passing in.
'trim_messages' helper to reduce how many messages we're sending to the model. The trimmer allows us to specify how many tokens we want to keep, along with other parameters like if we want to always keep the system message and whether to allow partial messages

In [25]:
from langchain_core.messages import SystemMessage,trim_messages
trimmer=trim_messages(
    max_tokens=45,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human"
)
messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]
trimmer.invoke(messages)

[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [26]:
from operator import itemgetter

from langchain_core.runnables import RunnablePassthrough
# RunnablePassthrough: Pass the input forward unchanged.
# assign: let me add or overwrite some keys.
# itemgetter("messages"): At runtime, extracts input["messages"]

# At runtime, it takes the input dictionary, pulls out messages, 
# trims them, and puts the trimmed messages back into the input under 
# the same key (messages), while leaving everything else unchanged.
chain=(
    RunnablePassthrough.assign(messages=itemgetter("messages")|trimmer)
    | prompt
    | model
    
)

response=chain.invoke(
    {
    "messages":messages + [HumanMessage(content="What ice cream do i like")],
    "language":"English"
    }
)
response.content

"I don't have any information about your preferences, so I'm not sure what ice cream you like. Would you like to tell me?"

In [27]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what math problem did i ask")],
        "language": "English",
    }
)
response.content

'You asked me a simple math problem: 2 + 2.'

In [28]:
## Lets wrap this in the MEssage History
# input_messages_key: It tells LangChain that the user’s messages are located under the key messages in the input dictionary.
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)
config={"configurable":{"session_id":"chat5"}}

In [29]:
response = with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="whats my name?")],
        "language": "English",
    },
    config=config,
)

response.content

"You didn't mention your name earlier, so I don't know what it is. Would you like to tell me?"

In [30]:
response = with_message_history.invoke(
    {
        "messages": [HumanMessage(content="what math problem did i ask?")],
        "language": "English",
    },
    config=config,
)

response.content

"This conversation has just started, and you haven't asked a math problem yet. I'm here to help, so please go ahead and ask your math question, and I'll do my best to assist you."